In [58]:
import pandas as pd
import numpy as np
import json, os, string
from janitor import clean_names

In [43]:
def read_json_dict(path: str) -> dict:
    """
    Reads a json file and returns it as dict object
    """
    
    file = open(path) # Opening JSON file
    return json.load(file) # returns JSON object as a dictionary

def folder_creator(folder_name: string, path: string) -> None:
    """
    Generates a folder in specified path
    
    input: name of root folder, path where you want 
    folder to be created
    output: None
    """
    
    # defining paths
    data_folder_path = path + "/" + folder_name
    data_folder_exists = os.path.exists(data_folder_path)

    # creating folders if don't exist
    if data_folder_exists:
        pass
    else:    
        # create a new directory because it does not exist 
        os.makedirs(data_folder_path)

        # create subfolders
        print(f"The new directory {folder_name} was created!")

## Reading paths

In [18]:
paths = read_json_dict("paths.json")

In [19]:
paths

{'data_path': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice',
 'code_path': '/Users/brandonmora/GitHub/peru-amag-stats/case_outcomes',
 'data_amag_i': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/01_AMAG',
 'data_cej': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/data_cleaned',
 'data_gender': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/07_Other/02_Raw/names_gender',
 'local_storage': 'D:/Daniel Chen Dropbox/Marco Antonio GutiÃ©rrez ChÃ¡vez/datasets_amag_ii_scrape'}

In [20]:
data_path = paths["data_path"]

In [42]:
folder_creator("data_cleaned", data_path)

The new directory data_cleaned is created!


In [44]:
data_cleaned_path = data_path + "/data_cleaned"

In [45]:
folder_creator("raw", data_cleaned_path)

The new directory raw was created!


In [46]:
dc_raw_path = data_cleaned_path + "/raw"

In [52]:
dc_raw_path

'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/data_cleaned/raw'

# 1. Creating participants list

In [30]:
lab_data = pd.read_stata(data_path + "/data/lab_Data/Clean_Full_Data12.dta")

In [48]:
lab_data["participant_nombre_apellido"] = lab_data["Nombres"] + " " + lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"]
lab_data["participant_nombre_apellido"] = lab_data["participant_nombre_apellido"].str.strip()

In [49]:
lab_data["participant_apellido_nombre"] = lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"] + " " + lab_data["Nombres"]
lab_data["participant_apellido_nombre"] = lab_data["participant_apellido_nombre"].str.strip()

In [50]:
lab_data = lab_data.rename(columns={"DNI": "nrodocumento"})

In [51]:
lab_data[["nrodocumento", "participant_nombre_apellido", "participant_apellido_nombre"]].to_csv(dc_raw_path + "/amag_ii_participants_list.csv")

# 2. Creating Cases List

### Selecting reporte files

In [59]:
files_reports = pd.read_csv(dc_raw_path + "/DF_file_report_2022.csv")
files_reports = clean_names(files_reports)

### Cleaning the reporte files

Creating lists with characters to be replace

In [65]:
backslash_reps = ["\\(\\*\\)", "\\", "\\([^()]{0,}\\)"]
trailing_and_special_reps = ["^\\s", "\\,", "\\.$", " \\- JUZ$", "\\*"]
other_strs_reps = ["\\- MIXTO Y LIQ", "\\- MIXTO", "\\- JUZ\\. MIXTO", 
                   "- JM", "- INVESTIGACION", "- PAZ LETRADO", "SECOM - ", "- JT"]

In [67]:
empty_reps = backslash_reps + trailing_and_special_reps +  other_strs_reps

In [66]:
name_reps = [["ALFREDO E\\.", "ALFREDO E"], ["BERTHA F\\.", "BERTHA F"], ["CLAUDIO W\\.", "CLAUDIO W"], 
            ["CLAVELITO L\\.", "CLAVELITO L"], ["ELMER L\\.", "ELMER L"], ["ERNESTO A\\.", "ERNESTO A"],
            ["HERBERT M\\.", "HERBERT M"], ["LUZ K\\.", "LUZ K"], ["NANCY S\\.", "NANCY S"], ["JESSICA E\\.", "JESSICA E"],
            ["PATRICIA C\\.", "PATRICIA C"], ["JESSICA P\\.", "JESSICA P"], ["YOLANDA B\\.", "YOLANDA B\\."],
            ["LUZ M\\.", "LUZ M"], ["EDGAR\\.", "EDGAR"], ["C\\. ARTURO", "C ARTURO"], ["ALEXANDER A\\.", "ALEXANDER A"],
            ["RENE G\\.", "RENE G"], ["GUILLERMO S\\.", "GUILLERMO S"], ["FANNY L\\. ",  "FANNY L"], ["ELISA \\(LA", "ELISA"],
            ["JULIA \\(LA", "JULIA"], ["ACEVEDO DIEZ CECILIA", "ACEVEDO DIEZ CECILIA DEL PILAR"]]

In [68]:
for val in empty_reps:    
    files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_36852\3422139958.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_36852\3422139958.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")


In [70]:
for name_rep in name_reps:
    files_reports["juez_"] = files_reports["juez_"].str.replace(name_rep[0], name_rep[1])

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_36852\1927001137.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  files_reports["juez_"] = files_reports["juez_"].str.replace(name_rep[0], name_rep[1])
